In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search
)

In [3]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-07-12 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-07-12 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-07-12 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-07-12 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-07-12 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(13551, 384)

# Baseline: FFN using signatures

First, we dimension reduce these and then take signatures. We use the path signature as input to the FFN for classification.

We want to choose a dimension and signature depth such that the number of terms in the signature is _roughly_ 384 so that it is comparable to the number of features that we used for the previous baseline where we computed the mean of the history. Again, we are concatenating the features we obtain with the current utterance embedding.

In [7]:
num_epochs = 100
hidden_dim_sizes = [[128,128],[256,256],[512,512]]
dropout_rates = [0.5, 0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [8]:
dim_reduce_methods = ["gaussian_random_projection", "umap"]

## Using log signature

In [9]:
log_signature_dimensions_and_sig_depths = [(28, 2), (10, 3), (6, 4)]

In [10]:
import signatory

[signatory.logsignature_channels(channels, depth)
 for (channels, depth) in log_signature_dimensions_and_sig_depths]

[406, 385, 406]

### Using UMAP

In [11]:
ffn_logsignature_umap_kfold, best_ffn_logsignature_umap_kfold, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    log_signature=True,
    dim_reduce_methods=["umap"],
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_umap_focal_{gamma}_kfold_min_dist_099_n_neighbours_50.csv",
    verbose=False
)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_logsignature_umap_focal_2_kfold_min_dist_099_n_neighbours_50.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_logsignature_umap_focal_2_kfold_min_dist_099_n_neighbours_50_best_model.csv


In [12]:
ffn_logsignature_umap_kfold.groupby(["dimension","sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_60951/3138393360.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_logsignature_umap_kfold.groupby(["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate                       
6         4         (128, 128) 0.1          0.0001         0.668937  0.593474   
                                            0.0005         0.673963  0.593186   
                                            0.0010         0.670097  0.580967   
                               0.2          0.0001         0.664119  0.582986   
                                            0.0005         0.665993  0.572468   
...                                                             ...       ...   
28        2         (512, 512) 0.2          0.0005         0.689219  0.610864   
                                            0.0010         0.688268  0.608980   
                               0.5          0.0001         0.701442  0.628646   
                                            0.0005         0.700223  0.615804   
                                            0.0010         0.691926  0.591098   

                                                           precision  \
dimension sig_depth hidden_dim dropout_rate learning_rate              
6         4         (128, 128) 0.1          0.0001          0.584959   
                                            0.0005          0.589686   
                                            0.0010          0.585125   
                               0.2          0.0001          0.579044   
                                            0.0005          0.581978   
...                                                              ...   
28        2         (512, 512) 0.2          0.0005          0.603542   
                                            0.0010          0.602928   
                               0.5          0.0001          0.619392   
                                            0.0005          0.614364   
                                            0.0010          0.603088   

                                                             recall  \
dimension sig_depth hidden_dim dropout_rate learning_rate             
6         4         (128, 128) 0.1          0.0001         0.605465   
                                            0.0005         0.598471   
                                            0.0010         0.579193   
                               0.2          0.0001         0.591532   
                                            0.0005         0.567244   
...                                                             ...   
28        2         (512, 512) 0.2          0.0005         0.620701   
                                            0.0010         0.617252   
                               0.5          0.0001         0.641227   
                                            0.0005         0.618774   
                                            0.0010         0.583870   

                                                           valid_accuracy  \
dimension sig_depth hidden_dim dropout_rate learning_rate                   
6         4         (128, 128) 0.1          0.0001               0.670338   
                                            0.0005               0.675450   
                                            0.0010               0.674910   
                               0.2          0.0001               0.667207   
                                            0.0005               0.666577   
...                                                                   ...   
28        2         (512, 512) 0.2          0.0005               0.690923   
                                            0.0010               0.684099   
                               0.5          0.0001               0.700743   
                                            0.0005               0.696306   
                                            0.0010               0.689640   

                                                           valid_f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate             


In [13]:
best_ffn_logsignature_umap_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,loss_function,gamma,k_fold,n_splits,batch_size,input_dim,dimension,sig_depth,method,log_signature
0,None,0.702751,0.633733,"[0.7893248062968884, 0.6066101199181047, 0.505...",0.621647,"[0.8198773633111905, 0.5922330097087378, 0.452...",0.651366,"[0.7609675124496087, 0.6217026378896883, 0.571...",None,0.705518,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.703197,0.625713,"[0.7921368765926465, 0.6062554808535515, 0.478...",0.618056,"[0.8111332007952287, 0.5915573302909298, 0.451...",0.635079,"[0.7740099596869813, 0.6217026378896883, 0.509...",None,0.709572,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.706320,0.630052,"[0.7946924004825091, 0.5972141613464887, 0.498...",0.625346,"[0.8087404861281611, 0.5787401574803149, 0.488...",0.635455,"[0.7811240218164572, 0.6169064748201439, 0.508...",None,0.713176,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.711822,0.630503,"[0.8011750881316098, 0.587138863000932, 0.5031...",0.631618,"[0.7940833915676683, 0.6092843326885881, 0.491...",0.630139,"[0.8083945933127816, 0.5665467625899281, 0.515...",None,0.716667,...,focal,2,True,5,64,790,28,2,umap,True
0,None,0.708253,0.630953,"[0.7972044824677672, 0.6068633353401566, 0.488...",0.623930,"[0.8103870651641353, 0.6094316807738815, 0.451...",0.640301,"[0.7844439174768792, 0.60431654676259, 0.53214...",None,0.708333,...,focal,2,True,5,64,790,28,2,umap,True


In [14]:
best_ffn_logsignature_umap_kfold["f1"].mean()

0.6301908793348754

In [15]:
best_ffn_logsignature_umap_kfold["precision"].mean()

0.6241194658310414

In [16]:
best_ffn_logsignature_umap_kfold["recall"].mean()

0.6384679884399672

In [17]:
np.stack(best_ffn_logsignature_umap_kfold["f1_scores"]).mean(axis=0)

array([0.79490673, 0.60081639, 0.49484952])

In [18]:
np.stack(best_ffn_logsignature_umap_kfold["precision_scores"]).mean(axis=0)

array([0.8088443 , 0.5962493 , 0.46726479])

In [19]:
np.stack(best_ffn_logsignature_umap_kfold["recall_scores"]).mean(axis=0)

array([0.781788  , 0.60623501, 0.52738095])

### Using random projections

In [20]:
ffn_logsignature_grp_kfold, best_ffn_logsignature_grp_kfold, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    log_signature=True,
    dim_reduce_methods=["gaussian_random_projection"],
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_logsignature_grp_focal_{gamma}_kfold.csv",
    verbose=False
)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/13551 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_logsignature_grp_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_logsignature_grp_focal_2_kfold_best_model.csv


In [21]:
ffn_logsignature_grp_kfold.groupby(["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_8227/2536377135.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_logsignature_grp_kfold.groupby(["dimension", "sig_depth", "hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate                       
6         4         (128, 128) 0.1          0.0001         0.719792  0.651365   
                                            0.0005         0.712595  0.647273   
                                            0.0010         0.709651  0.641849   
                               0.2          0.0001         0.720208  0.652678   
                                            0.0005         0.712922  0.646207   
...                                                             ...       ...   
28        2         (512, 512) 0.2          0.0005         0.720565  0.658076   
                                            0.0010         0.720922  0.654281   
                               0.5          0.0001         0.733859  0.669734   
                                            0.0005         0.729338  0.665236   
                                            0.0010         0.725978  0.659365   

                                                           precision  \
dimension sig_depth hidden_dim dropout_rate learning_rate              
6         4         (128, 128) 0.1          0.0001          0.641685   
                                            0.0005          0.634828   
                                            0.0010          0.629609   
                               0.2          0.0001          0.642415   
                                            0.0005          0.634773   
...                                                              ...   
28        2         (512, 512) 0.2          0.0005          0.644177   
                                            0.0010          0.642935   
                               0.5          0.0001          0.659195   
                                            0.0005          0.654031   
                                            0.0010          0.649071   

                                                             recall  \
dimension sig_depth hidden_dim dropout_rate learning_rate             
6         4         (128, 128) 0.1          0.0001         0.665781   
                                            0.0005         0.668674   
                                            0.0010         0.662427   
                               0.2          0.0001         0.667355   
                                            0.0005         0.665586   
...                                                             ...   
28        2         (512, 512) 0.2          0.0005         0.678371   
                                            0.0010         0.669841   
                               0.5          0.0001         0.684078   
                                            0.0005         0.681416   
                                            0.0010         0.673067   

                                                           valid_accuracy  \
dimension sig_depth hidden_dim dropout_rate learning_rate                   
6         4         (128, 128) 0.1          0.0001               0.726059   
                                            0.0005               0.718514   
                                            0.0010               0.716306   
                               0.2          0.0001               0.728378   
                                            0.0005               0.720968   
...                                                                   ...   
28        2         (512, 512) 0.2          0.0005               0.724302   
                                            0.0010               0.723694   
                               0.5          0.0001               0.736622   
                                            0.0005               0.731644   
                                            0.0010               0.729775   

                                                           valid_f1  \
dimension sig_depth hidden_dim dropout_rate learning_rate             


In [22]:
best_ffn_logsignature_grp_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,loss_function,gamma,k_fold,n_splits,batch_size,input_dim,dimension,sig_depth,method,log_signature
0,None,0.736654,0.669513,"[0.817209470015623, 0.6320267395928288, 0.5593...",0.661426,"[0.8284600389863548, 0.640788662969809, 0.5150...",0.680555,"[0.8062603746739389, 0.6235011990407674, 0.611...",None,0.739077,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.734870,0.670863,"[0.8141528800869461, 0.6308113035551504, 0.567...",0.660841,"[0.8294783464566929, 0.6395563770794824, 0.513...",0.685403,"[0.7993834479487788, 0.6223021582733813, 0.634...",None,0.736937,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.735465,0.673377,"[0.8128420284567675, 0.6390708755211436, 0.568...",0.661732,"[0.8342486270594108, 0.6349112426035503, 0.516...",0.689312,"[0.7925065212236186, 0.6432853717026379, 0.632...",None,0.738063,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.732342,0.669212,"[0.8109555690809496, 0.6342469790745653, 0.562...",0.657967,"[0.8331665832916458, 0.623768115942029, 0.5169...",0.683883,"[0.7898980317761441, 0.645083932853717, 0.6166...",None,0.736937,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True
0,None,0.729963,0.665707,"[0.8094249786350872, 0.63556338028169, 0.55213...",0.654009,"[0.8341721187720181, 0.6224137931034482, 0.505...",0.681239,"[0.7861038653070903, 0.6492805755395683, 0.608...",None,0.732095,...,focal,2,True,5,64,790,28,2,gaussian_random_projection,True


In [23]:
best_ffn_logsignature_grp_kfold["f1"].mean()

0.6697344898298014

In [24]:
best_ffn_logsignature_grp_kfold["precision"].mean()

0.6591948435897377

In [25]:
best_ffn_logsignature_grp_kfold["recall"].mean()

0.6840784604607382

In [26]:
np.stack(best_ffn_logsignature_grp_kfold["f1_scores"]).mean(axis=0)

array([0.81291699, 0.63434386, 0.56194263])

In [27]:
np.stack(best_ffn_logsignature_grp_kfold["precision_scores"]).mean(axis=0)

array([0.83190514, 0.63228764, 0.51339175])

In [28]:
np.stack(best_ffn_logsignature_grp_kfold["recall_scores"]).mean(axis=0)

array([0.79483045, 0.63669065, 0.62071429])